In [1]:
import os
import pandas as pd
import numpy as np
import pickle

from model_with_p_values import LinearRegressionWithPValues, LogisticRegressionWithPValues

In [2]:
pd.options.display.max_rows = None
# pd.options.display.max_columns = None

In [3]:
data_folder = "C:/Users/Hrushikesh.Hrushikesh-Asus/Desktop/Courses/Udemy/Credit Risk Modeling in Python/dataset/"

# Use os.path.join to concatenate the folder and file paths
file_path = os.path.join(data_folder, 'loan_data_2007_2014_preprocessed.csv')

# Read the CSV file with the index_col parameter specified
loan_data_preprocessed_backup = pd.read_csv(file_path, index_col=0, low_memory=False)

In [4]:
input_train_path = os.path.join(data_folder, 'loan_data_inputs_train.csv')
loan_data_inputs_train = pd.read_csv(input_train_path, index_col=0)

input_test_path = os.path.join(data_folder, 'loan_data_inputs_test.csv')
loan_data_inputs_test = pd.read_csv(input_test_path, index_col=0)

## Load the previous models

In [5]:
model_folder = "C:/Users/Hrushikesh.Hrushikesh-Asus/Desktop/Courses/Udemy/Credit Risk Modeling in Python/mywork/model/"

with open(os.path.join(model_folder, 'lgd_model_stage_1.sav'), 'rb') as file:
    reg_lgd_st_1 = pickle.load(file)

with open(os.path.join(model_folder, 'lgd_model_stage_2.sav'), 'rb') as file:
    reg_lgd_st_2 = pickle.load(file)

with open(os.path.join(model_folder, 'ead_model.sav'), 'rb') as file:
    reg_ead = pickle.load(file)

with open(os.path.join(model_folder, 'pd_model.sav'), 'rb') as file:
    reg_pd = pickle.load(file)

In [6]:
loan_data_preprocessed = loan_data_preprocessed_backup.copy()
loan_data_preprocessed.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,...,0,0,0,0,0,0,0,0,1,0
1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,...,0,0,0,0,0,0,0,0,1,0
1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,0,0,0,0,0,0,0,0,1,0
1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,0,0,0,0,0,0,0,0,1,0
1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,...,0,0,0,0,0,0,0,0,1,0


In [7]:
loan_data_preprocessed.shape

(466285, 205)

In [8]:
loan_data_preprocessed['mths_since_last_delinq'].fillna(0, inplace = True)
loan_data_preprocessed['mths_since_last_record'].fillna(0, inplace = True)

In [9]:
features_all = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']

In [10]:
features_reference_cat = ['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

In [11]:
loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]
loan_data_preprocessed_lgd_ead = loan_data_preprocessed_lgd_ead.drop(features_reference_cat, axis = 1)

In [12]:
loan_data_preprocessed['recovery_rate_st_1'] = reg_lgd_st_1.model.predict(loan_data_preprocessed_lgd_ead)
loan_data_preprocessed['recovery_rate_st_2'] = reg_lgd_st_2.predict(loan_data_preprocessed_lgd_ead)

In [13]:
loan_data_preprocessed['recovery_rate'] = loan_data_preprocessed['recovery_rate_st_1'] * loan_data_preprocessed['recovery_rate_st_2']

In [14]:
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] < 0, 0, loan_data_preprocessed['recovery_rate'])
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] > 1, 1, loan_data_preprocessed['recovery_rate'])

In [15]:
loan_data_preprocessed['LGD'] = 1 - loan_data_preprocessed['recovery_rate']

In [16]:
loan_data_preprocessed['LGD'].describe()

count    466285.000000
mean          0.932203
std           0.058047
min           0.359266
25%           0.880789
50%           0.913580
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

In [17]:
loan_data_preprocessed['CCF'] = reg_ead.predict(loan_data_preprocessed_lgd_ead)

In [18]:
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] < 0, 0, loan_data_preprocessed['CCF'])
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] > 1, 1, loan_data_preprocessed['CCF'])

In [19]:
loan_data_preprocessed['EAD'] = loan_data_preprocessed['CCF'] * loan_data_preprocessed_lgd_ead['funded_amnt']

In [20]:
loan_data_preprocessed['EAD'].describe()

count    466285.000000
mean      10815.538277
std        6935.541913
min         189.310017
25%        5496.091958
50%        9209.894201
75%       14693.620178
max       35000.000000
Name: EAD, dtype: float64

In [21]:
loan_data_preprocessed.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,...,0,0,1,0,1,0.086608,0.086608,0.913392,0.591017,2955.083712
1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,...,0,0,1,0,1,0.085006,0.085006,0.914994,0.779061,1947.653143
1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,0,0,1,0,1,0.080869,0.080869,0.919131,0.659317,1582.361773
1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,0,0,1,0,1,0.095469,0.095469,0.904531,0.661756,6617.559269
1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,...,0,0,1,0,1,0.088950,0.088950,0.911050,0.709287,2127.862253


In [22]:
loan_data_inputs_train.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86
id,,,,,,,,,,,,,,,,,,,,,
12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,A5,Supervisor inventory management,...,1,0,0,1,0,0,0,0,0,0
1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,C1,mizuho corporate bank,...,0,0,0,1,0,0,0,0,0,0
5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,A5,Internal Medicine of Griffin,...,0,0,0,1,0,0,0,0,0,0
13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,D1,Administrative Assistant,...,0,0,0,1,0,0,0,0,0,0
422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,C2,Bank of A,...,0,1,0,1,0,0,0,0,0,0


In [23]:
loan_data_inputs_test.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86
id,,,,,,,,,,,,,,,,,,,,,
19677589,21900299,32500,32500,32500.0,60 months,14.99,773.01,C,C5,Licensed vocational nurse,...,0,0,0,1,0,0,0,0,0,0
29755527,32278795,11000,11000,11000.0,60 months,20.99,297.53,E,E4,shift manager,...,0,1,0,1,0,0,0,0,0,0
1343334,1588314,30000,30000,30000.0,36 months,14.65,1034.83,C,C2,FastMed urgent care,...,0,0,0,1,0,0,0,0,0,0
33131681,35775007,14400,14400,14400.0,60 months,14.49,338.74,C,C4,Registered Nurse,...,0,0,0,1,0,0,0,0,0,0
3293168,4066358,15000,15000,14900.0,36 months,8.90,476.30,A,A5,Courier Times,...,0,0,0,1,0,0,0,0,0,0


In [24]:
loan_data_inputs_train.shape, loan_data_inputs_test.shape

((373028, 321), (93257, 321))

In [25]:
loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)
loan_data_inputs_pd.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86
id,,,,,,,,,,,,,,,,,,,,,
12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,A5,Supervisor inventory management,...,1,0,0,1,0,0,0,0,0,0
1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,C1,mizuho corporate bank,...,0,0,0,1,0,0,0,0,0,0
5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,A5,Internal Medicine of Griffin,...,0,0,0,1,0,0,0,0,0,0
13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,D1,Administrative Assistant,...,0,0,0,1,0,0,0,0,0,0
422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,C2,Bank of A,...,0,1,0,1,0,0,0,0,0,0


In [26]:
loan_data_inputs_pd.shape

(466285, 321)

In [27]:
features_all_pd = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<40',
'mths_since_issue_d:40-42',
'mths_since_issue_d:42-46',
'mths_since_issue_d:46-52',
'mths_since_issue_d:52-56',
'mths_since_issue_d:56-70',
'mths_since_issue_d:>70',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>=86']

In [28]:
ref_categories_pd = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>70',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [29]:
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]
loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd, axis = 1)

In [30]:
loan_data_inputs_pd_temp.shape

(466285, 83)

In [31]:
loan_data_inputs_pd['PD'] = reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]

In [32]:
loan_data_inputs_pd['PD'].head()

id
12796369    0.029665
1439740     0.110731
5214749     0.036341
13827698    0.201229
422455      0.206721
Name: PD, dtype: float64

In [33]:
loan_data_inputs_pd['PD'].describe()

count    466285.000000
mean          0.109309
std           0.070800
min           0.007538
25%           0.056108
50%           0.093536
75%           0.146563
max           0.644331
Name: PD, dtype: float64

In [34]:
loan_data_inputs_pd.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86,PD
id,,,,,,,,,,,,,,,,,,,,,
12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,A5,Supervisor inventory management,...,0,0,1,0,0,0,0,0,0,0.029665
1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,C1,mizuho corporate bank,...,0,0,1,0,0,0,0,0,0,0.110731
5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,A5,Internal Medicine of Griffin,...,0,0,1,0,0,0,0,0,0,0.036341
13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,D1,Administrative Assistant,...,0,0,1,0,0,0,0,0,0,0.201229
422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,C2,Bank of A,...,1,0,1,0,0,0,0,0,0,0.206721


In [35]:
loan_data_preprocessed.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,...,0,0,1,0,1,0.086608,0.086608,0.913392,0.591017,2955.083712
1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,...,0,0,1,0,1,0.085006,0.085006,0.914994,0.779061,1947.653143
1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,0,0,1,0,1,0.080869,0.080869,0.919131,0.659317,1582.361773
1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,0,0,1,0,1,0.095469,0.095469,0.904531,0.661756,6617.559269
1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,...,0,0,1,0,1,0.088950,0.088950,0.911050,0.709287,2127.862253


In [36]:
loan_data_preprocessed_new = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)

In [37]:
loan_data_preprocessed_new.shape

(466285, 533)

In [38]:
def get_first_occurrence(lst):
    first_occurrence = {}
    output = []
    
    for idx, ele in enumerate(lst):
        if ele not in first_occurrence:
            first_occurrence[ele] = idx
            output.append(idx)
    
    return output

loan_data_preprocessed_new = loan_data_preprocessed_new.iloc[:, get_first_occurrence(loan_data_preprocessed_new.columns)]

In [39]:
loan_data_preprocessed_new.shape

(466285, 328)

In [40]:
loan_data_preprocessed_new[['PD', 'LGD', 'EAD']].sample(10)

,PD,LGD,EAD
id,,,
12458134,0.016948,0.876811,6643.734892
5946151,0.047523,0.888202,3114.186050
11995961,0.236267,0.856930,7086.851172
16081696,0.130616,0.860490,8695.382022
10120891,0.062669,0.868720,12974.605061
11384819,0.081071,0.867119,21608.743355
1623599,0.117749,0.888415,14839.637282
22474662,0.029568,0.864601,14994.339042
9010042,0.081822,1.000000,7249.438782


In [41]:
loan_data_preprocessed_new['EL'] = loan_data_preprocessed_new['PD'] * loan_data_preprocessed_new['LGD'] * loan_data_preprocessed_new['EAD']
# We calculate Expected Loss. EL = PD * LGD * EAD.

In [42]:
loan_data_preprocessed_new['EL'].describe()

count    466285.000000
mean       1093.651034
std        1108.049587
min           8.823142
25%         358.376248
50%         714.592434
75%        1421.812884
max       11635.169238
Name: EL, dtype: float64

In [43]:
loan_data_preprocessed_new[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head()

,funded_amnt,PD,LGD,EAD,EL
id,,,,,
1077501,5000,0.163446,0.913392,2955.083712,441.164197
1077430,2500,0.276904,0.914994,1947.653143,493.468488
1077175,2400,0.227895,0.919131,1582.361773,331.449907
1076863,10000,0.199137,0.904531,6617.559269,1191.990990
1075358,3000,0.123327,0.911050,2127.862253,239.081092


In [44]:
loan_data_preprocessed_new['funded_amnt'].describe()

count    466285.000000
mean      14291.801044
std        8274.371300
min         500.000000
25%        8000.000000
50%       12000.000000
75%       20000.000000
max       35000.000000
Name: funded_amnt, dtype: float64

In [45]:
loan_data_preprocessed_new['EL'].sum()

509953072.2770806

In [46]:
loan_data_preprocessed_new['funded_amnt'].sum()

6664052450

In [47]:
loan_data_preprocessed_new['EL'].sum() / loan_data_preprocessed_new['funded_amnt'].sum()

0.07652296798430519